# Transfer Learning

In [15]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import numpy as np
import matplotlib.pyplot as plt

In [2]:
model = tf.keras.applications.vgg16.VGG16(include_top=False, input_shape = (32,32,3))

58889256/58889256 [==============================] - 5s 0us/step


In [3]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
X_train = tf.keras.applications.vgg16.preprocess_input(X_train)
X_test = tf.keras.applications.vgg16.preprocess_input(X_test)
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

170498071/170498071 [==============================] - 13s 0us/step


In [4]:
model.trainable = False
input_layer = model.input
x = model(input_layer)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dense(10, activation = 'softmax')(x)

In [5]:
new_model = tf.keras.models.Model(inputs = input_layer, outputs = x)
new_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dense_1 (Dense)             (None, 10)                2570      
                                                                 
Total params: 14,848,586
Trainable params: 133,898
Non-trainable params: 14,714,688
_________________________________________________________________


In [6]:
new_model.compile(loss ='categorical_crossentropy', optimizer = 'adam', metrics = ['acc'])

In [7]:
history = new_model.fit(X_train, y_train, validation_split = 0.1, epochs = 5)

Epoch 1/5
1407/1407 [==============================] - 24s 12ms/step - loss: 1.9292 - acc: 0.5608 - val_loss: 1.0961 - val_acc: 0.6288
Epoch 2/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.9682 - acc: 0.6659 - val_loss: 1.0574 - val_acc: 0.6422
Epoch 3/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.8593 - acc: 0.6995 - val_loss: 1.0645 - val_acc: 0.6486
Epoch 4/5
1407/1407 [==============================] - 16s 11ms/step - loss: 0.7909 - acc: 0.7255 - val_loss: 1.1297 - val_acc: 0.6436
Epoch 5/5
1407/1407 [==============================] - 14s 10ms/step - loss: 0.7189 - acc: 0.7504 - val_loss: 1.1214 - val_acc: 0.6550


In [8]:
new_model.evaluate(X_test, y_test)

313/313 [==============================] - 3s 10ms/step - loss: 1.1783 - acc: 0.6372


[1.1782855987548828, 0.6371999979019165]

# MobileNet & ShuffleNet

## MobileNet

In [16]:
def bottleneck_block(x, expand=96, squeeze=16):
  m = layers.Conv2D(expand, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(x)
  m = layers.BatchNormalization()(m)
  m = layers.Activation('relu')(m)
  m = layers.DepthwiseConv2D((3,3), padding='same', kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
  m = layers.BatchNormalization()(m)
  m = layers.Activation('relu')(m)
  m = layers.Conv2D(squeeze, (1,1), kernel_regularizer=regularizers.l2(0.0001), use_bias = False)(m)
  m = layers.BatchNormalization()(m)
  return layers.Add()([m, x])

In [17]:
input = layers.Input(shape=(32, 32, 3))
x = layers.Conv2D(16, 1, activation='relu', padding='same')(input)
for i in range (5):
  x = bottleneck_block(x)
flatten = layers.Flatten()(x)
dense = layers.Dense(10, activation="softmax")(flatten)
model = tf.keras.models.Model(inputs=input, outputs=dense)

In [18]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [19]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(1e-3), loss='categorical_crossentropy', metrics=['acc'])

In [20]:
model.fit(x_train, y_train, batch_size=64, epochs=1, validation_split=0.2)

625/625 [==============================] - 30s 41ms/step - loss: 2.0087 - acc: 0.4667 - val_loss: 2.0622 - val_acc: 0.3638


## ShuffleNet

In [22]:
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, MaxPool2D, GlobalAveragePooling2D, Reshape, Permute, Lambda, Flatten, Activation
from tensorflow import keras

In [ ]:
def getModel():
  input = keras.Input(shape=(19, 19, planes), name='board')
  x = Conv2D(trunk, 1, padding='same', kernel_regularizer=regularizers.l2(0.0001))(input)
  x = BatchNormalization()(x)
  x = ReLU()(x)
  for i in range (blocks):
    x = bottleneck_block (x, filters, trunk)
  policy_head = Conv2D(1, 1, activation='relu', padding='same', use_bias = False, kernel_regularizer=regularizers.l2(0.0001))(x)
  policy_head = Flatten()(policy_head)
  policy_head = Activation('softmax', name='policy')(policy_head)
  value_head = GlobalAveragePooling2D()(x)
  value_head = Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(value_head)
  value_head = Dense(1, activation='sigmoid', name='value', kernel_regularizer=regularizers.l2(0.0001))(value_head)
  model = keras.Model(inputs=input, outputs=[policy_head, value_head])
  return model